In [8]:
import os
import re
import sys
import toml
import json
from tqdm import tqdm

In [9]:
root = os.getcwd().split('notebooks')[0]

In [10]:
# Get data and inference ids
inference_ids = ["grwmh_greenshields_model_greenshields_sim_learn_noise_n200",
                  "grwmh_daganzos_model_daganzos_sim_learn_noise_n200",
                  "grwmh_delcastillos_model_delcastillos_sim_learn_noise_n200",
                  "grwmh_greenbergs_model_greenbergs_sim_learn_noise_n200",
                  "grwmh_underwoods_model_underwoods_sim_learn_noise_n200",
                  "grwmh_northwesterns_model_northwesterns_sim_learn_noise_n200",
                  "grwmh_newells_model_newells_sim_learn_noise_n200",
                  "grwmh_wangs_model_wangs_sim_learn_noise_n200",
                  "grwmh_smulders_model_smulders_sim_learn_noise_n200",
                  "grwmh_deromphs_model_deromphs_sim_learn_noise"]
data_ids = ["greenshields_fd_log_simulation_n200",
            "daganzos_fd_log_simulation_n200",
            "delcastillos_fd_log_simulation_n200",
            "greenbergs_fd_log_simulation_n200",
            "underwoods_fd_log_simulation_n200",
            "northwesterns_fd_log_simulation_n200",
            "newells_fd_log_simulation_n200",
            "wangs_fd_log_simulation_n200",
            "smulders_fd_log_simulation_n200",
            "deromphs_fd_log_simulation_n200"]

In [17]:
# Proposal adjustments for mcmc tuning
proposal_adjustments = { 
    "daganzos": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.6},
    "delcastillos": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.6},
    "deromphs": {"vanilla_mcmc":0.8,"thermodynamic_integration_mcmc":0.3},
    "greenbergs": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.5},
    "greenshields": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.5},
    "newells": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.5},
    "northwesterns": {"vanilla_mcmc":0.7,"thermodynamic_integration_mcmc":0.3},
    "smulders": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.6},
    "underwoods": {"vanilla_mcmc":0.7,"thermodynamic_integration_mcmc":0.3},
    "wangs": {"vanilla_mcmc":0.9,"thermodynamic_integration_mcmc":0.4}
}

## Create simulation parameters for new dataset size

In [19]:
# New dataset size
new_n = 500
# Flag for exporting parameters to file
export = False

In [20]:
for data_id in tqdm(data_ids):
    
    # Import toml file
    simulation_filename = os.path.join(root,'data','input','simulation_parameters',(data_id+'.toml'))
    if os.path.exists(simulation_filename):
        simulation_parameters  = toml.load(simulation_filename)
    
    # Ammend dataset size in id and rho_steps
    simulation_parameters['id'] = simulation_parameters['id'].replace("_n200",f"_n{str(new_n)}")
    simulation_parameters['rho']['interval1']['rho_steps'] = int(new_n)
    
    # Get new simulation filename
    new_simulation_filename = simulation_filename.replace("_n200",f"_n{str(new_n)}")

    # Export
    if export:
        with open(new_simulation_filename, 'w') as f:
            new_toml_string = toml.dump(simulation_parameters, f)
    
    

100%|██████████| 10/10 [00:00<00:00, 743.93it/s]


## Create inference parameters for marginal likelihood comparisons
This refers to creating copies of an inference id for running inference on different datasets

In [38]:
# Pick inference id index
index = 4
# Flag for exporting parameters to file
export = False

# Make sure index is in list
assert index < len(inference_ids)

# Get inference id
inference_id = inference_ids[index]

inference_id

'grwmh_underwoods_model_underwoods_sim_learn_noise_n200'

In [37]:
# Loop through datasets
for data_id in tqdm(data_ids):
        
    # Get dataset FD model name
    data_fd = data_id.split('_fd',1)[0]
    model_fd = inference_id.split('_model',1)[0].split("_",1)[1]
    
#     print('data_fd:',data_fd,'model_fd:',model_fd)
    
    # Replace simulation dataset iff model_fd != data_fd
    if data_fd != model_fd: 
        new_inference_id = re.sub(r'\_model.+\_sim',f'_model_{data_fd}_sim',inference_id)
    else: continue
        
    # Import toml file
    inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,(inference_id+'.toml'))
    if os.path.exists(inference_filename):
        inference_parameters = toml.load(inference_filename)
    
    # Change id and data fundamental diagram properties
    inference_parameters['id'] = new_inference_id
    inference_parameters['data_id'] = re.sub(r'.+\_fd',f'{data_fd}_fd',inference_parameters['data_id'])
    inference_parameters['data_fundamental_diagram'] = data_fd
        
    # Change proposal adjustments
    inference_parameters['inference']['vanilla_mcmc']['transition_kernel']['adapt_proposal_during_burnin'] = "True"
    inference_parameters['inference']['vanilla_mcmc']['transition_kernel']['proposal_factor_adjustment'] = float(proposal_adjustments[data_fd]['vanilla_mcmc'])
    inference_parameters['inference']['thermodynamic_integration_mcmc']['transition_kernel']['adapt_proposal_during_burnin'] = "True"
    inference_parameters['inference']['thermodynamic_integration_mcmc']['transition_kernel']['proposal_factor_adjustment'] = float(proposal_adjustments[data_fd]['thermodynamic_integration_mcmc'])
    
#     print(new_inference_id)
#     print(json.dumps(inference_parameters,indent=2))
    # Export
    if export:
        new_inference_filename = os.path.join(root,'data','input','inference_parameters',model_fd,(new_inference_id+'.toml'))
        with open(new_inference_filename, 'w') as f:
            new_toml_string = toml.dump(inference_parameters, f)
    
#     sys.exit(1)
    

100%|██████████| 10/10 [00:00<00:00, 173.96it/s]
